In [ ]:
import re
import json
import numpy as np
import pandas as pd
from copy import deepcopy
#Account Age
import time
from datetime import datetime

#SQL
import mysql.connector 
from mysql.connector import errorcode
import config
import json

import matplotlib.pyplot as plt

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

## Product Analysis 

1. look into trending products
2. Find average price of trending products 

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['Successful', 'sale', 'to', 'Sold', 'sold', 'an', 'Bought', 'a', 'from', 'purchased', 'some', 'cash', 'for', 'with', 'was', 'the', 'bought', 'it', 'is', 'Purchased', 'trade', 'Traded', 'great', 'new', 
 'shipping', 'received', 'my', 'of', 'supply', 'in', 'me', 'Confirmed', 'very', 'off', 'Great', 'broken', 'shipped',  
 'traded', 'as', 'I', 'set', 'parts', 'on', 'locally', 'you', 'but', 'Thanks', 'Thanks!', 'Thank', 'used', ']', '[', '(', ')', ' Local Cash', ' PayPal', '  Paypal', ' Local', ' local cash', '  PayPal', ' Paypal', ' Cash', ' Local cash', ' cash', ' paypal', '  Local Cash', 'Trades', 'PayPal', 'Local', 'Cash', 'Paypal', 'local', ' paypal', 'Verified', ' Steam Link'] )

In [ ]:
# Pulls all submissions from SQL AWS server and creates dataframe 

def sql_to_df():
    cnx = mysql.connector.connect(
        host = config.host,
        user = config.user,
        password = config.passwd,
        database = config.db_name
    )
    cur = cnx.cursor()
    cur.execute("""SELECT * FROM Submissions.submissions;""")
    df_sql = pd.DataFrame(cur.fetchall())
    df_sql.columns = [x[0] for x in cur.description]
    cur.close()
    cnx.close()
    return df_sql

In [ ]:
df = sql_to_df()